In [ ]:
import param
import panel as pn

pn.extension()

When building custom applications and dashboards it is frequently useful to extend Panel with custom components, which are specific to a particular application. Panel provides multiple mechanisms to extend and compose diffferent components or even add entirely new components. In this user guide we will go over these approaches and compare the benefits and drawbacks.

## Viewer components

The simplest way to extend Panel is to implement so called `Viewer` components. These components simply wrap other Panel object and make it possible to compose them as a unit just like any native Panel component. The core mechanism that makes this possible is the implementation of a ``__panel__`` method on the class, which Panel will call when displaying the component.

Below we will declare a composite `EditableRange` component made up of two `FloatInput` widgets. The class creates the widgets and then sets up callbacks to sync the parameters on the underlying widgets with the parameters on the `Viewer` component and then implements the ``__panel__`` method, which returns the Panel layout to be rendered when displaying the component:

In [ ]:
from panel.viewable import Viewer

class EditableRange(Viewer):
    
    value = param.Range(doc="A numeric range.")
    
    width = param.Integer(default=300)
    
    def __init__(self, **params):
        self._start_input = pn.widgets.FloatInput()
        self._end_input = pn.widgets.FloatInput(align='end')
        super().__init__(**params)
        self._layout = pn.Row(self._start_input, self._end_input)
        self._sync_widgets()
    
    def __panel__(self):
        return self._layout
    
    @param.depends('value', 'width', watch=True)
    def _sync_widgets(self):
        self._start_input.name = self.name
        self._start_input.value = self.value[0]
        self._end_input.value = self.value[1]
        self._start_input.width = self.width//2
        self._end_input.width = self.width//2
        
    @param.depends('_start_input.value', '_end_input.value', watch=True)
    def _sync_params(self):
        self.value = (self._start_input.value, self._end_input.value)
    
range_widget = EditableRange(name='Range', value=(0, 10))

pn.Column(
    '## This is a custom widget',
    range_widget
)

Implementing a component by subclassing the `Viewer` baseclass gives the component a number of useful affordances:
    
* It renders itself in a notebook (like all other Panel components)
* It can be placed in a Panel layout component (such as a `Row` or `Column`)
* It has `show` and `servable` methods

This approach is very helpful when we want to wrap multiple existing Panel components into a easily reusable unit.

## ReactiveHTML components

The `ReactiveHTML` provides bi-directional syncing of arbitrary HTML attributes and DOM properties with parameters on the subclass. This kind of component must declare a few class-attributes which declare 

- `_html`: The HTML template to render declaring how to link parameters on the class to HTML attributes.
- `_dom_events` (optional): Optional mapping of named nodes to DOM events to add event listeners to.
- `_scripts` (optional): Optional mapping of Javascript to execute on specific parameter changes.

### HTML templates

A ReactiveHTML component is declared by providing an HTML template on the `_html` attribute on the class. Parameters are synced by inserting them as template variables of the form `${parameter}`, e.g.:

```html
    _html = '<div class="${div_class}">${children}</div>'
```

will interpolate the div_class parameter on the `class` attribute of the HTML element. In addition to providing attributes we can also provide children to an HTML tag. Any child parameter will be treated as other Panel components to render into the containing HTML. This makes it possible to use `ReactiveHTML` to lay out other components.

### DOM Events

In certain cases it is necessary to explicitly declare event listeners on the DOM node to ensure that changes in their properties are synced when an event is fired. To make this possible the HTML element in question must be given a unique id, e.g.:

```html
    _html = '<input id="input"></input>'
```

Now we can use this name to declare set of `_dom_events` to subscribe to. The following will subscribe to change DOM events on the input element:

```python
    _dom_events = {'input': ['change']}
```

Once subscribed the class may also define a method following the `_{node}_{event}` naming convention which will fire when the DOM event triggers, e.g. we could define a `_input_change` method. Any such callback will be given a DOMEvent object as the first and only argument. The DOMEvent contains information about the event on the .data attribute and declares the type of event on the .type attribute.

### Inline callbacks
    
Instead of declaring explicit DOM events Python callbacks can also be declared inline, e.g.:

```html
    _html = '<input id="input" onchange="${_input_change}"></input>'
```

will look for an `_input_change` method on the `ReactiveHTML` component and call it when the event is fired.

### Scripts
    
In addition to declaring callbacks in Python it is also possible to declare Javascript callbacks to execute when any sync attribute changes. Let us say we have declared an input element with a synced value parameter:

```html
    _html = '<input id="input" value="${value}"></input>'
```

We can now declare a set of `_scripts`, which will fire whenever the value updates:

```python
   _scripts = {
     'value': ['console.log(model, data, state, input)']
   }
```

The Javascript is provided multiple objects in its namespace including:


* `data`:  The data model holds the current values of the synced parameters, e.g. data.value will reflect the current value of the input node.
* `model`:  The ReactiveHTML model which holds layout information and information about the children and events.
* `state`:  An empty state dictionary which scripts can use to store state for the lifetime of the view.
* `<node>`: All named DOM nodes in the HTML template, e.g. the `input` node in the example above.

### Example

To see all of this in action we declare a `Slideshow` component which subscribes to `click` events on an `<img>` element and advances the image `index` on each click:

In [ ]:
from panel.reactive import ReactiveHTML

class Slideshow(ReactiveHTML):
    
    index = param.Integer(default=0)
    
    _html = '<img id="img" src="https://picsum.photos/800/300?image=${index}"></img>'

    _scripts = {
        'index': ['console.log(data.index, img)']
    }

    _dom_events = {
        'img': ['click']
    }
    
    def _img_click(self, event):
        self.index += 1
        
Slideshow(width=800, height=300)

As we can see this approach lets us quickly build custom HTML components with complex interactivity.

## Building custom bokeh models

The last approach to extending Panel with new components is to write custom Bokeh models. This involves writing, compiling and distributing custom Javascript and therefore requires considerably more effort than the other approaches. Detailed documentation on writing such components will be coming to the developer guide in the future.